In [1]:
import requests
import csv
import re
import json
import urllib.request
from bs4 import BeautifulSoup

for vano in range(2017, 2021):
    for vmes in range(1, 13):
        ano=str(vano)
        mes=""
        if vmes<10 :
            mes="0"         
        mes+=str(vmes)
        mesano=mes+ano
        anomes=ano+mes
        aux ="https://sisgvarmazenamento.blob.core.windows.net/prd/PublicacaoPortal/Arquivos/"
        aux+=anomes       
        aux+=".htm"
        #Consulte o site e retorne o html para a variável 'page'
        page = urllib.request.urlopen(aux)
        soup = BeautifulSoup(page, 'html5lib')


        # Criar um arquivo para gravar, adicionar linha de cabeçalhos
        f = csv.writer(open(('Dados'+mesano+'.csv'), 'w'))
        f.writerow(['Vereador','Tipo_de_Gasto','Nome_Da_Empresa','CNPJ','Valor','Mes/Ano'])
        jsonFilePath=('Dados'+mesano+'.json')
        jsondata = [] 
       
        


    
        #Regex para remover tags html
        TAG_RE = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
        def remove_tags(text):
            return TAG_RE.sub('', text)

        #Inicialização de variaveis
        index=0
        start=0
        nomeVereador=""
        categoriaDespesa=""
        cnpj=""
        LugarDespesa=""
        skip=0
        ignore=0
        bugduplo=1
        #Percorre buscando Tds
        name_list = soup.find('body')
        name_list2 = soup.find(class_="bloco")
        for tr in name_list.find_all('tr'):
            if bugduplo==0:
                bugduplo=-1
                continue
            if(tr.find(text=re.compile("[\s\S]+(Vereador)\\((a)\\)[:\s]",re.I))):
                start=1
                name_list_itemsp = tr.find_all('td')
                for name in name_list_itemsp:
                    if(name.find(text=re.compile("[\s\S]+(Vereador)\\((a)\\)",re.I))):
                        names = str(name.contents[1])
                        names=re.sub("[\s\S]+(Vereador)\\((a)\\)[:\s]","",names)
                        names=remove_tags(names)
                        nomeVereador=names
                        ignore=0
                        if bugduplo==1:
                            bugduplo=0
            if(start!=0):
                name_list_itemsv = tr.find_all('td')
                for name in name_list_itemsv:                   
                    names = str(name.contents[0])
                    if(skip==1):
                        skip=0
                        continue
                    if(ignore==1):
                        continue
                    names=remove_tags(names)
                    names=re.sub("(Natureza da despesa)","",names)
                    names=re.sub("(Valor utilizado)","",names)                    
                    names=re.sub("(VALORES GASTOS)","",names)
                    names=re.sub("(VALORES DISPONIBILIZADOS)","",names)
                    names=re.sub("(TOTAL DO ITEM)","VXASkip",names)
                    names=re.sub("(TOTAL DO MÊS)","VXBSkip",names)
                    names=re.sub("(VEREADOR AFASTADO)","VXBSkip",names)
                    if re.match("\d{2}.?\d{3}.?\d{3}/?\d{4}-?\d{2}", names) is not None:              #RegexCNPJ
                        start=2
                    if re.match("[\s\S]*(VXASkip)", names) is not None:              #Corte Aqui  
                        start=1;
                        skip=1;
                        continue 
                    if re.match("[\s\S]*(VXBSkip)", names) is not None:              #Corte Aqui  
                        start=0;
                        ignore=1
                        break 
                    if re.match(r'^\s*$', names):
                        continue
                    if(start==1):
                        categoriaDespesa=names;
                        start=2;
                        continue            
                    if(start==2):   
                        cnpj=names;
                        start=3
                        continue
                    if(start==3):   
                        LugarDespesa=names;
                        start=4
                        continue    
                    print(nomeVereador+", "+categoriaDespesa+", "+LugarDespesa+", "+cnpj+", "+names+", "+mes+"/"+ano)
                    if(start>=4):
                        f.writerow([nomeVereador,categoriaDespesa,LugarDespesa,cnpj,names,(mes+"/"+ano)])              
                        content={}
                        content['Vereador'] = nomeVereador
                        content['Tipo_de_Gasto'] = categoriaDespesa
                        content['Nome_Da_Empresa'] = LugarDespesa
                        content['CNPJ'] = cnpj
                        content['Valor'] = names
                        content['Mes/Ano'] = (mes+"/"+ano)
                        jsondata.append(content)
                        start=2;
                    start+=1
        #Ainda dentro do for de mes
        with open(jsonFilePath, 'w') as outfile:
            json.dump(jsondata, outfile)

 ABOU ANNI, COMBUSTIVEL, PROTOTIPO AUTO POSTO LTDA., 43.900.851/0001-01, 880,00, 01/2017
 ABOU ANNI, INTERMEDIADO - CORREIOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 5.993,27, 01/2017
 ABOU ANNI, MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE CONSUMO, CENTER PAPEIS COMERCIAL LTDA., 06.226.820/0001-82, 219,45, 01/2017
 ABOU ANNI, TELEFONE MOVEL, TELEFONICA BRASIL S/A, 02.558.157/0001-62, 266,10, 01/2017
 ADILSON AMADEU, ELABORAÇÃO/MANUTENÇAO DE SITE/HOSPEDAGEM, GENERICAPONTOCOM SERVIÇOS DE INTERNET LTDA, 11.913.587/0001-46, 5.946,00, 01/2017
 ADILSON AMADEU, INTERMEDIADO - CORREIOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 1.892,84, 01/2017
 ADILSON AMADEU, INTERMEDIADO - LOCAÇÃO DE VEÍCULOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 2.058,08, 01/2017
 ADILSON AMADEU, TELEFONE FIXO, TELEFONICA BRASIL S/A, 02.558.157/0001-62, 198,20, 01/2017
 ADRIANA RAMALHO, ASSINATURA DE JORNAIS E REVISTAS, ABRIL COMUNICAÇÕES S/A, 44.597.052/0079-22, 780,00, 01/2017
 ADRIANA

 EDUARDO MATARAZZO SUPLICY, ASSINATURA DE JORNAIS E REVISTAS, EDITORA CONFIANÇA LTDA., 04.735.670/0001-07, 699,00, 01/2017
 EDUARDO MATARAZZO SUPLICY, ASSINATURA DE JORNAIS E REVISTAS, EDITORA CAROS AMIGOS LTDA, 08.254.606/0001-65, 104,40, 01/2017
 EDUARDO MATARAZZO SUPLICY, ASSINATURA DE JORNAIS E REVISTAS, PALAVRA LIVRE, 14.620.529/0001-12, 150,00, 01/2017
 EDUARDO MATARAZZO SUPLICY, ASSINATURA DE JORNAIS E REVISTAS, INFOGLOBO COMUNICAÇÕES E PARTIC SA, 60.452.752/0029-16, 805,90, 01/2017
 EDUARDO MATARAZZO SUPLICY, ASSINATURA DE JORNAIS E REVISTAS, S.A O ESTADO DE S. PAULO, 61.533.949/0001-41, 658,80, 01/2017
 EDUARDO MATARAZZO SUPLICY, COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA, WBL GRAFICA E EDITORA LTDA, 08.142.850/0001-36, 120,00, 01/2017
 EDUARDO MATARAZZO SUPLICY, COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA, FELIPE SPENCER NOBREGA CABRAL, 24.992.302/0001-51, 2.973,00, 01/2017
 EDUARDO MATARAZZO SUPLICY, CONTRATAÇAO DE PESSOA JURIDICA, SOLARYS SOLUÇÕES DIG

 OTA, COMBUSTIVEL, M MARTINEZ COMBUSTIVEIS ME, 10.785.686/0001-27, 497,23, 01/2017
 OTA, COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA, IRISARA COMÉRCIO E SERVIÇOS LTDA., 10.376.277/0001-77, 268,38, 01/2017
 OTA, CONTRATAÇAO DE PESSOA JURIDICA, KLEMP , DE PAULA SOCIEDADE DE ADVOGADOS, 18.160.749/0001-61, 6.000,00, 01/2017
 OTA, CONTRATAÇAO DE PESSOA JURIDICA, WORK LINE SYSTEM INFORMÁTICA LTDA, 67.002.329/0001-62, 931,54, 01/2017
 OTA, INTERMEDIADO - CORREIOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 8.841,31, 01/2017
 OTA, INTERMEDIADO - LOCAÇÃO DE VEÍCULOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 2.058,08, 01/2017
 OTA, MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE CONSUMO, CENTER PAPEIS COMERCIAL LTDA., 06.226.820/0001-82, 439,80, 01/2017
 OTA, MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE CONSUMO, IRISARA COMÉRCIO E SERVIÇOS LTDA., 10.376.277/0001-77, 221,40, 01/2017
 OTA, TELEFONE MOVEL, NEXTEL TELECOMUNICAÇÕES LTDA, 66.970.229/0001-67, 190,99, 01/2017
 

 ABOU ANNI, COMBUSTIVEL, POSTO DE SERVIÇOS 23 DE MAIO LTDA., 18.811.994/0001-91, 150,00, 02/2017
 ABOU ANNI, COMBUSTIVEL, PROTOTIPO AUTO POSTO LTDA., 43.900.851/0001-01, 600,00, 02/2017
 ABOU ANNI, INTERMEDIADO - CORREIOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 6.053,87, 02/2017
 ABOU ANNI, MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE CONSUMO, IRISARA COMÉRCIO E SERVIÇOS LTDA., 10.376.277/0001-77, 1.099,00, 02/2017
 ABOU ANNI, TELEFONE MOVEL, TELEFONICA BRASIL S/A, 02.558.157/0001-62, 197,35, 02/2017
 ADILSON AMADEU, COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA, PHUTURA INOVAÇÕES GRAFICAS LTDA-ME, 02.540.850/0001-08, 7.952,00, 02/2017
 ADILSON AMADEU, INTERMEDIADO - CORREIOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 759,19, 02/2017
 ADILSON AMADEU, INTERMEDIADO - LOCAÇÃO DE VEÍCULOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 2.101,17, 02/2017
 ADILSON AMADEU, MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE CONSUMO, CENTER PAPEIS COMERCIAL LTDA., 06

 DALTON SILVANO, COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA, FORMA CERTA GRAFICA  DIGITAL LTDA, 07.691.829/0001-27, 7.862,54, 02/2017
 DALTON SILVANO, CONTRATAÇAO DE PESSOA JURIDICA, LOCAWEB IDC LTDA, 06.887.099/0001-71, 381,47, 02/2017
 DALTON SILVANO, CONTRATAÇAO DE PESSOA JURIDICA, WORK LINE SYSTEM INFORMÁTICA LTDA, 67.002.329/0001-62, 931,54, 02/2017
 DALTON SILVANO, ELABORAÇÃO/MANUTENÇAO DE SITE/HOSPEDAGEM, UAU DIGITAL COMUNICAÇÃO LTDA, 10.202.058/0001-71, 6.698,00, 02/2017
 DALTON SILVANO, INTERMEDIADO - LOCAÇÃO DE VEÍCULOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 2.101,17, 02/2017
 DALTON SILVANO, LOCAÇÃO DE MOVEIS E EQUIPAMENTOS, IRISARA COMÉRCIO E SERVIÇOS LTDA., 10.376.277/0001-77, 400,00, 02/2017
 DALTON SILVANO, MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE CONSUMO, IRISARA COMÉRCIO E SERVIÇOS LTDA., 10.376.277/0001-77, 585,00, 02/2017
 DALTON SILVANO, TELEFONE FIXO, TELEFONICA BRASIL S/A, 02.558.157/0001-62, 270,93, 02/2017
 DAVID SOARES, COMBUSTIVEL,

...

 ADILSON AMADEU, INTERMEDIADO - CORREIOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 22.629,39, 12/2019
 ADILSON AMADEU, INTERMEDIADO - LOCAÇÃO DE VEÍCULOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 2.214,62, 12/2019
 ADRIANA RAMALHO, ASSINATURA DE JORNAIS E REVISTAS, S.A O ESTADO DE S. PAULO, 61.533.949/0001-41, 102,90, 12/2019
 ADRIANA RAMALHO, COMBUSTIVEL, POSTO JENNER LTDA, 61.699.179/0001-01, 166,00, 12/2019
 ADRIANA RAMALHO, COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA, PLENAPRINT GRAFICA E EDITORA EIRELI, 14.674.740/0001-18, 1.310,00, 12/2019
 ADRIANA RAMALHO, COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA, NATALIA HERNANDEZ PEREIRA DE JESUS-ME, 23.506.925/0001-04, 5.851,00, 12/2019
 ADRIANA RAMALHO, CONTRATAÇAO DE PESSOA JURIDICA, LOCAWEB SERVIÇOS DE INTERNET S.A., 02.351.877/0001-52, 65,00, 12/2019
 ADRIANA RAMALHO, CONTRATAÇAO DE PESSOA JURIDICA, CONDE e DELL ARINGA - DESENVOLVIMENTO DE SOFTWARES LTDA - EPP, 09.613.772/0001-73, 250,00, 12/20

 DANIEL ANNENBERG, COMBUSTIVEL, M22 AUTO POSTO E CONVENIENCIA LTDA, 27.746.048/0001-08, 100,00, 12/2019
 DANIEL ANNENBERG, CONTRATAÇAO DE PESSOA JURIDICA, GIGOVI COMUNICAÇÃO LTDA, 31.237.594/0001-82, 8.000,00, 12/2019
 DANIEL ANNENBERG, ELABORAÇÃO/MANUTENÇAO DE SITE/HOSPEDAGEM, WEBSIDE SERVIÇOS EMPR LTDA ME, 08.652.665/0001-91, 9.000,00, 12/2019
 DANIEL ANNENBERG, INTERMEDIADO - LOCAÇÃO DE VEÍCULOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 1.476,41, 12/2019
 DANIEL ANNENBERG, MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE CONSUMO, K2G SUPRIMENTOS DE IMPRESSÃO E INFORMATICA EIRELLI - ME, 19.367.661/0001-88, 2.071,00, 12/2019
 DONATO, COMBUSTIVEL, POSTO JUPIA LT, 02.183.065/0001-45, 150,01, 12/2019
 DONATO, COMBUSTIVEL, POSTO ENERGIA AUTOMOTIVA LTDA, 19.499.974/0001-90, 535,19, 12/2019
 DONATO, COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA, DUBLEGRAF GRAFICA E EDITORA LTDA ME, 05.883.576/0001-68, 11.100,00, 12/2019
 DONATO, CONTRATAÇAO DE PESSOA JURIDICA, IP3 CONSULTORIA

 JOSÉ POLICE NETO, APLICATIVO OTTCS - DESLOCAMENTO, 99 TECNOLOGIA LTDA, 18.033.552/0001-61, 1.454,26, 12/2019
 JOSÉ POLICE NETO, COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA, RWC - GRÁFICA E EDITORA LTDA ME, 24.827.045/0001-00, 3.250,00, 12/2019
 JOSÉ POLICE NETO, CONTRATAÇAO DE PESSOA JURIDICA, RD GESTÃO E SISTEMAS S.A., 13.021.784/0001-86, 1.254,48, 12/2019
 JOSÉ POLICE NETO, INTERMEDIADO - CORREIOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 14,20, 12/2019
 JOSÉ POLICE NETO, LOCAÇÃO DE MOVEIS E EQUIPAMENTOS, FMJ COMÉRCIO E ASSISTENCIA TÉCNICA EM COPIADORAS LTDA ME, 07.915.380/0001-33, 852,50, 12/2019
 JOSÉ POLICE NETO, TELEFONE MOVEL, TELEFONICA BRASIL S/A, 02.558.157/0001-62, 429,99, 12/2019
 JULIANA CARDOSO, COMBUSTIVEL, POSTO DE SERVIÇOS CCS LTDA, 52.661.501/0001-12, 438,54, 12/2019
 JULIANA CARDOSO, COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA, AZ ARTES GRAFICAS E EDITORA LTDA, 00.893.009/0001-79, 3.770,00, 12/2019
 JULIANA CARDOSO, COMPOSIÇÃO/ARTE/DI

 SANDRA TADEU, COMBUSTIVEL, AUTO POSTO OSCAR PORTO LTDA, 47.682.034/0001-77, 300,08, 12/2019
 SANDRA TADEU, COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA, G 7 GRÁFICA E EDITORA LTDA ., 68.919.935/0001-00, 7.150,00, 12/2019
 SANDRA TADEU, CONTRATAÇAO DE PESSOA JURIDICA, SISNETTI INTEGRADORA DE SOLUÇÕES LTDA. ME, 01.319.851/0001-64, 620,00, 12/2019
 SANDRA TADEU, CONTRATAÇAO DE PESSOA JURIDICA, AGENCIA IDEAL COMUNICAÇÃO LTDA, 07.479.559/0001-95, 10.000,00, 12/2019
 SANDRA TADEU, INTERMEDIADO - CORREIOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 21.527,91, 12/2019
 SANDRA TADEU, INTERMEDIADO - LOCAÇÃO DE VEÍCULOS, CAMARA MUNICIPAL DE SÃO PAULO, 50.176.288/0001-28, 2.214,62, 12/2019
 SANDRA TADEU, MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE CONSUMO, K2G SUPRIMENTOS DE IMPRESSÃO E INFORMATICA EIRELLI - ME, 19.367.661/0001-88, 110,00, 12/2019
 SANDRA TADEU, TELEFONE FIXO, TELEFONICA BRASIL S/A, 02.558.157/0001-62, 378,91, 12/2019
 SANDRA TADEU, TELEFONE MOVEL, TELEFONICA B

HTTPError: HTTP Error 404: The specified blob does not exist.